# Randomized Benchmarking: Unitarity RB

In [ ]:
# See rb_functions in examples folder for related usage
# Needs in terminal:
# $ quilc -S
# $ qvm -S

from forest.benchmarking.analysis.fitting import make_figure
from forest.benchmarking.randomized_benchmarking import *

from pyquil.api import get_benchmarker
import numpy as np
from typing import List, Tuple, Callable
from pandas import DataFrame, Series
from pyquil import Program, get_qc
from pyquil import noise

## Add functionality to inject understandable error into RB sequence

In [ ]:
def add_noise_to_sequences(sequences, qubits, kraus_ops):
    """
    Append the given noise to each clifford gate (sequence)
    """
    for seq in sequences:
        for program in seq:
            program.defgate("noise", np.eye(2 ** len(qubits)))
            program.define_noisy_gate("noise", qubits, kraus_ops)
            program.inst(("noise", *qubits))
    
def depolarizing_noise(num_qubits: int, p: float =.95):
    """
    Generate the Kraus operators corresponding to a given unitary
    single qubit gate followed by a depolarizing noise channel.

    :params float num_qubits: either 1 or 2 qubit channel supported
    :params float p: parameter in depolarizing channel as defined by: p $\rho$ + (1-p)/d I
    :return: A list, eg. [k0, k1, k2, k3], of the Kraus operators that parametrize the map.
    :rtype: list
    """
    num_of_operators = 4**num_qubits
    probabilities = [p+(1.0-p)/num_of_operators] + [(1.0 - p)/num_of_operators]*(num_of_operators-1)
    return noise.pauli_kraus_map(probabilities)

bm = get_benchmarker()
# get an ideal and noisy qc
qc = get_qc("9q-square-qvm", noisy=False)
noisy_qc = get_qc("4q-qvm", noisy=True)

## Run unitarity RB (This is SLOW)

In [ ]:
qubits = (0,) # e.g. (3,4) for two qubits

single_clifford_p = .95 #p parameter for the depolarizing channel applied to each clifford
kraus_ops = depolarizing_noise(len(qubits), single_clifford_p)

num_sequences_per_depth = 50
num_shots = 25
depths = 2 * 2 ** np.arange(4, dtype=np.uint8) # depth = number of cliffords in a sequence
depths = [d for d in depths for _ in range(num_sequences_per_depth)]


sequences = generate_rb_experiment_sequences(bm, qubits, depths, use_self_inv_seqs=False)
add_noise_to_sequences(sequences, qubits, kraus_ops)

expts = group_sequences_into_parallel_experiments([sequences], [qubits], is_unitarity_expt=True)

results = acquire_rb_data(qc, expts, num_shots)

stats = get_stats_by_qubit_group([qubits], results)[qubits]
fit = fit_unitarity_results(depths, stats['expectation'], stats['std_err'])

# plot the raw data, point estimate error bars, and fit
fig, axs = make_figure(fit, xlabel="Sequence Length [Cliffords]", ylabel="Shifted Purity")

In [ ]:
unitarity = fit.params['decay'].value
print(unitarity)
err = fit.params['decay'].stderr
print(err)

In [ ]:
from forest.benchmarking.randomized_benchmarking import unitarity_to_rb_decay
# Since noise is depolarizing, we expect this value to match the 
# input noise parameter single_clifford_p = .95 
print(unitarity_to_rb_decay(unitarity, 2))
print(unitarity_to_rb_decay(unitarity-err, 2))
print(unitarity_to_rb_decay(unitarity+err, 2))

## If we don't need to alter the seqeuences (e.g. we don't care too much about a specific noise model or are running on a QPU) we can directly generate experiments. It is also easier to generate simulatneous unitarity experiments.

In [ ]:
qubit_groups = [(0,),(2,),(3,)]
expts = generate_unitarity_experiments(bm, qubit_groups, depths)

results = acquire_rb_data(noisy_qc, expts, num_shots)
stats_by_group = get_stats_by_qubit_group(qubit_groups, results)
fits = []
for group in qubit_groups:
    stats = stats_by_group[group]
    fits.append(fit_unitarity_results(depths, stats['expectation'], stats['std_err']))

In [ ]:
for fit, group in zip(fits, qubit_groups):
    fig, axs = make_figure(fit, xlabel="Sequence Length [Cliffords]", ylabel="Shifted Purity", 
                           title='Qubit' + str(group[0]))